In [5]:
# ! pip install numpy==1.26.4

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Параметры
num_radars = 16
num_trajectories = 100
input_features = ['x_measure', 'v_x_measure_extr', 'r_measure', 'fi_measure', 'psi_measure']
target_feature = 'x_true'
data_path = 'data/'

# Функция для загрузки данных одной траектории
def load_trajectory_data(traj_idx):
    trajectory_data = []
    for radar_idx in range(1, num_radars + 1):
        file_path = os.path.join(data_path, f'traj{traj_idx}_radar{radar_idx}.csv')
        df = pd.read_csv(file_path)
        trajectory_data.append(df)
    
    # Объединение данных по временной оси
    traj_data = pd.concat(trajectory_data, axis=0).reset_index(drop=True)
    return traj_data

# Подготовка данных
X_data, y_data = [], []

for traj_idx in range(1, num_trajectories + 1):
    data = load_trajectory_data(traj_idx)
    X = data[input_features].values  # Входные данные
    y = data[target_feature].values  # Истинная координата
    X_data.append(X)
    y_data.append(y)

# Конвертируем в numpy массивы
X_data = np.array(X_data)  # (num_trajectories, seq_length, num_features)
y_data = np.array(y_data)  # (num_trajectories, seq_length)

# Заменяем NaN на 0 и Inf на большое число
X_data = np.nan_to_num(X_data, nan=0.0, posinf=1e6, neginf=-1e6)
y_data = np.nan_to_num(y_data, nan=0.0, posinf=1e6, neginf=-1e6)


# Делим на тренировочную и тестовую выборки
train_size = int(0.8 * num_trajectories)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

# Кастомный Dataset для PyTorch
class RadarDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Создаем DataLoader
batch_size = 1  # Один батч — одна траектория
train_dataset = RadarDataset(X_train, y_train)
test_dataset = RadarDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


C:\Users\lamin\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\lamin\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # Выходной слой для регрессии

    def forward(self, x):
        # LSTM: возвращает все временные шаги и последний скрытый слой
        out, (hidden, _) = self.lstm(x)
        # Выбираем последнее состояние и пропускаем через линейный слой
        return self.fc(hidden[-1])

# Параметры модели
input_size = len(input_features)  # Число входных признаков
hidden_size = 64
num_layers = 2

model = LSTMModel(input_size, hidden_size, num_layers)


In [5]:
# Оптимизатор и функция потерь
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("device: ", device)

# Обучение
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Обнуление градиентов
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(X_batch)
        outputs = outputs.squeeze(-1)  # Убираем последнюю размерность для совместимости

        # Проверяем размерность целевого тензора
        if len(y_batch.shape) > 1:
            y_batch = y_batch[:, -1]  # Берем последний временной шаг для истинного значения

        # Вычисляем потери
        loss = criterion(outputs, y_batch)

        # Обратный проход
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # Вычисляем среднее значение потерь за эпоху
    train_loss /= len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")


# Оценка на тестовых данных
model.eval()
test_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch[:, -1])  # Последний шаг
        test_loss += loss.item()

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")


device:  cpu
Epoch [1/100], Loss: 65310.1636
Epoch [2/100], Loss: 62085.6763
Epoch [3/100], Loss: 58659.6536
Epoch [4/100], Loss: 55790.7667
Epoch [5/100], Loss: 53297.2489
Epoch [6/100], Loss: 50992.2295
Epoch [7/100], Loss: 48817.4113
Epoch [8/100], Loss: 46750.3469
Epoch [9/100], Loss: 44772.7603
Epoch [10/100], Loss: 42881.0486
Epoch [11/100], Loss: 41060.8761
Epoch [12/100], Loss: 39306.3765
Epoch [13/100], Loss: 37619.2231
Epoch [14/100], Loss: 35993.9506
Epoch [15/100], Loss: 34424.0276
Epoch [16/100], Loss: 32914.9060
Epoch [17/100], Loss: 31458.2173
Epoch [18/100], Loss: 30057.4309
Epoch [19/100], Loss: 28708.0938
Epoch [20/100], Loss: 27409.0876
Epoch [21/100], Loss: 26160.7731
Epoch [22/100], Loss: 24957.3623
Epoch [23/100], Loss: 23801.0492
Epoch [24/100], Loss: 22686.8096
Epoch [25/100], Loss: 21621.5690
Epoch [26/100], Loss: 20596.7670
Epoch [27/100], Loss: 19607.3759
Epoch [28/100], Loss: 18663.9536
Epoch [29/100], Loss: 17758.7482
Epoch [30/100], Loss: 16885.9193
Epoch 

C:\Users\lamin\anaconda3\lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 2123.0420
